In [28]:
import pandas as pd

In [29]:
income_df = pd.read_csv("IRS.csv") 
income_df = income_df[['ZIPCODE','Adjusted gross income (AGI)','STATE']].rename(columns={"ZIPCODE": "Zip", "Adjusted gross income (AGI)": "AdjGrossIncome", "STATE": "State"})
income_df.head()

,Zip,AdjGrossIncome,State
0,35004,255534,AL
1,35005,128387,AL
2,35006,58302,AL
3,35007,643708,AL
4,35010,378497,AL


In [31]:
income_df.shape

(27688, 3)

In [32]:
zip_df = pd.read_csv("ZIPFPS.csv")
zip_df = zip_df[['ZIP','STCOUNTYFP','COUNTYNAME','STATE']].rename(columns={"ZIP": "Zip", "STCOUNTYFP": "FIPS", "COUNTYNAME": "CountyName", "STATE": "State"})
zip_df.head()

,Zip,FIPS,CountyName,State
0,36091,1001,Autauga County,AL
1,36758,1001,Autauga County,AL
2,36006,1001,Autauga County,AL
3,36067,1001,Autauga County,AL
4,36701,1001,Autauga County,AL


In [33]:
zip_df.shape

(53962, 4)

In [34]:
sample = income_df.merge(zip_df, on = ['Zip','State'], how = 'left')
sample.head()

,Zip,AdjGrossIncome,State,FIPS,CountyName
0,35004,255534,AL,1073.0,Jefferson County
1,35004,255534,AL,1115.0,St. Clair County
2,35005,128387,AL,1073.0,Jefferson County
3,35006,58302,AL,1073.0,Jefferson County
4,35006,58302,AL,1125.0,Tuscaloosa County


In [35]:
sample.shape

(41651, 5)

In [36]:
income_county = sample[['FIPS','AdjGrossIncome']].groupby('FIPS').mean().round(0).reset_index()
income_county

,FIPS,AdjGrossIncome
0,1001.0,183386.0
1,1003.0,240259.0
2,1005.0,55247.0
3,1007.0,129462.0
4,1009.0,186912.0
...,...,...
3114,56037.0,433566.0
3115,56039.0,315748.0
3116,56041.0,152509.0
3117,56043.0,128494.0


In [37]:
health_df = pd.read_csv("DIabetesByCounty.csv")
health_df = health_df[['State','County','CountyFIPS','Number']].rename(columns={"County": "County", "CountyFIPS": "FIPS", "Number": "DiabetesCount"})
health_df

,State,County,FIPS,DiabetesCount
0,Hawaii,Kalawao County,15005,10
1,Texas,King County,48269,20
2,Nebraska,Arthur County,31005,23
3,Nebraska,Blaine County,31009,28
4,Texas,Kenedy County,48261,32
...,...,...,...,...
3214,California,Orange County,6059,206411
3215,Arizona,Maricopa County,4013,276140
3216,Texas,Harris County,48201,322085
3217,Illinois,Cook County,17031,372183


In [38]:
final_data = income_county.merge(health_df, on = 'FIPS', how = 'left')
final_data

,FIPS,AdjGrossIncome,State,County,DiabetesCount
0,1001.0,183386.0,Alabama,Autauga County,5178
1,1003.0,240259.0,Alabama,Baldwin County,16707
2,1005.0,55247.0,Alabama,Barbour County,3400
3,1007.0,129462.0,Alabama,Bibb County,2371
4,1009.0,186912.0,Alabama,Blount County,6282
...,...,...,...,...,...
3114,56037.0,433566.0,Wyoming,Sweetwater County,2560
3115,56039.0,315748.0,Wyoming,Teton County,439
3116,56041.0,152509.0,Wyoming,Uinta County,1440
3117,56043.0,128494.0,Wyoming,Washakie County,637


In [39]:
final_data = final_data[['State','County','AdjGrossIncome','DiabetesCount']]
final_data

,State,County,AdjGrossIncome,DiabetesCount
0,Alabama,Autauga County,183386.0,5178
1,Alabama,Baldwin County,240259.0,16707
2,Alabama,Barbour County,55247.0,3400
3,Alabama,Bibb County,129462.0,2371
4,Alabama,Blount County,186912.0,6282
...,...,...,...,...
3114,Wyoming,Sweetwater County,433566.0,2560
3115,Wyoming,Teton County,315748.0,439
3116,Wyoming,Uinta County,152509.0,1440
3117,Wyoming,Washakie County,128494.0,637


In [161]:
# Import all required packages
import pandas as pd
import numpy as np
from google.cloud import bigquery as bq
from datetime import datetime, timedelta
import math

# #Initialize
client = bq.Client('geotab-bi') 

In [162]:
#Creating bigquery table from dataframe
dataset_id = "Project_Ramnath"
table_id = "CountyData"
table_ref = client.dataset(dataset_id).table(table_id)
job = client.load_table_from_dataframe(total_nona,table_ref)
job.result()

In [40]:
pop_2017_df = pd.read_csv("Pop2017.csv", skipinitialspace=True)
pop_modified=pop_2017_df[['state','county','population','proportion_of_white', 'proportion_of_black',
       'proportion_of_asian', 'proportion_of_indian', 'proportion_of_pacific',
       'proportion_of_two_or_more_races', 'proportion_of_non_hispanic',
       'proportion_of_hispanic']].rename(columns={"county": "County", "state": "State"})

In [41]:
pop_modified

,State,County,population,proportion_of_white,proportion_of_black,proportion_of_asian,proportion_of_indian,proportion_of_pacific,proportion_of_two_or_more_races,proportion_of_non_hispanic,proportion_of_hispanic
0,Alabama,Autauga,55390,0.768749,0.194530,0.012259,0.004640,0.001011,0.018812,0.971854,0.028146
1,Alabama,Baldwin,212521,0.873208,0.090589,0.010470,0.007684,0.000649,0.017401,0.954447,0.045553
2,Alabama,Barbour,25157,0.494932,0.481138,0.004730,0.006161,0.001749,0.011289,0.955917,0.044083
3,Alabama,Bullock,10176,0.263267,0.707154,0.001867,0.008156,0.007272,0.012284,0.920794,0.079206
4,Alabama,Butler,19888,0.524035,0.447858,0.013325,0.003671,0.000503,0.010609,0.986022,0.013978
...,...,...,...,...,...,...,...,...,...,...,...
3135,Wyoming,Sweetwater,43464,0.940410,0.013712,0.009594,0.014334,0.001403,0.020546,0.841501,0.158499
3136,Wyoming,Teton,23384,0.953515,0.006457,0.014583,0.008638,0.001368,0.015438,0.850197,0.149803
3137,Wyoming,Uinta,20431,0.954872,0.007978,0.004650,0.014341,0.001860,0.016299,0.908864,0.091136
3138,Wyoming,Washakie,8010,0.948814,0.005119,0.007615,0.016979,0.001124,0.020350,0.856929,0.143071


In [42]:
pop_2017_df = pop_2017_df[['state', 'county', 'year', 'population',
       'white_population', 'black_population', 'asian_population',
       'indian_population', 'pacific_population',
       'two_or_more_races_population']].rename(columns={"county": "County", "state": "State"})

In [43]:
final_data['County'] = final_data['County'].apply(lambda x: x.split(' ')[0])

/home/jupyter/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
final_data

,State,County,AdjGrossIncome,DiabetesCount
0,Alabama,Autauga,183386.0,5178
1,Alabama,Baldwin,240259.0,16707
2,Alabama,Barbour,55247.0,3400
3,Alabama,Bibb,129462.0,2371
4,Alabama,Blount,186912.0,6282
...,...,...,...,...
3114,Wyoming,Sweetwater,433566.0,2560
3115,Wyoming,Teton,315748.0,439
3116,Wyoming,Uinta,152509.0,1440
3117,Wyoming,Washakie,128494.0,637


In [45]:
pop_2017_df.columns

Index(['State', 'County', 'year', 'population', 'white_population',
       'black_population', 'asian_population', 'indian_population',
       'pacific_population', 'two_or_more_races_population'],
      dtype='object')

In [46]:
total_data = final_data.merge(pop_2017_df, on = ['County','State'], how = 'left')
total_data

,State,County,AdjGrossIncome,DiabetesCount,year,population,white_population,black_population,asian_population,indian_population,pacific_population,two_or_more_races_population
0,Alabama,Autauga,183386.0,5178,2017.0,55390.0,42581.0,10775.0,679.0,257.0,56.0,1042.0
1,Alabama,Baldwin,240259.0,16707,2017.0,212521.0,185575.0,19252.0,2225.0,1633.0,138.0,3698.0
2,Alabama,Barbour,55247.0,3400,2017.0,25157.0,12451.0,12104.0,119.0,155.0,44.0,284.0
3,Alabama,Bibb,129462.0,2371,2017.0,22550.0,17306.0,4844.0,44.0,93.0,26.0,237.0
4,Alabama,Blount,186912.0,6282,2017.0,57787.0,55445.0,938.0,170.0,367.0,64.0,803.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3114,Wyoming,Sweetwater,433566.0,2560,2017.0,43464.0,40874.0,596.0,417.0,623.0,61.0,893.0
3115,Wyoming,Teton,315748.0,439,2017.0,23384.0,22297.0,151.0,341.0,202.0,32.0,361.0
3116,Wyoming,Uinta,152509.0,1440,2017.0,20431.0,19509.0,163.0,95.0,293.0,38.0,333.0
3117,Wyoming,Washakie,128494.0,637,2017.0,8010.0,7600.0,41.0,61.0,136.0,9.0,163.0


In [47]:
total_data.columns

Index(['State', 'County', 'AdjGrossIncome', 'DiabetesCount', 'year',
       'population', 'white_population', 'black_population',
       'asian_population', 'indian_population', 'pacific_population',
       'two_or_more_races_population'],
      dtype='object')

In [48]:
total_data['Diabetesper1000']=((1000/total_data['population'])*total_data['DiabetesCount']).round(0)

In [49]:
total_data = total_data[['AdjGrossIncome', 'Diabetesper1000', 'white_population', 'black_population',
       'asian_population', 'indian_population', 'pacific_population',
       'two_or_more_races_population']]

In [50]:
total_data.isna().sum()

AdjGrossIncome                    0
Diabetesper1000                 317
white_population                317
black_population                317
asian_population                317
indian_population               317
pacific_population              317
two_or_more_races_population    317
dtype: int64

In [51]:
total_nona=total_data.dropna()

In [52]:
total_nona.isna().sum()

AdjGrossIncome                  0
Diabetesper1000                 0
white_population                0
black_population                0
asian_population                0
indian_population               0
pacific_population              0
two_or_more_races_population    0
dtype: int64

In [ ]:
# Split data training and test

In [54]:
pip install pycaret

     |████████████████████████████████| 266kB 11.7MB/s eta 0:00:01
ERROR: Could not find a version that satisfies the requirement pandas-profiling>=2.8.0 (from pycaret) (from versions: 1.2.0.linux-x86_64, 1.0.0a1, 1.0.0a2, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 2.0.3, 2.1.0, 2.1.2, 2.2.0, 2.3.0, 2.4.0, 2.5.0)
ERROR: No matching distribution found for pandas-profiling>=2.8.0 (from pycaret)
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# **Normalization train dataset**

In [26]:
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled
    
# call the maximum_absolute_scaling function
df_cars_scaled = maximum_absolute_scaling(total_nona)

df_cars_scaled

,AdjGrossIncome,Diabetesper1000,white_population,black_population,asian_population,indian_population,pacific_population,two_or_more_races_population
0,0.100771,0.407895,0.011810,0.008637,0.001021,0.002122,0.000594,0.004718
1,0.132023,0.346491,0.051471,0.015432,0.003347,0.013485,0.001465,0.016742
2,0.030358,0.592105,0.003453,0.009702,0.000179,0.001280,0.000467,0.001286
3,0.071140,0.460526,0.004800,0.003883,0.000066,0.000768,0.000276,0.001073
4,0.102708,0.478070,0.015378,0.000752,0.000256,0.003031,0.000679,0.003635
...,...,...,...,...,...,...,...,...
1319,0.029616,0.171053,0.005688,0.000082,0.000191,0.000347,0.000032,0.000896
1320,0.061222,0.342105,0.007963,0.000349,0.001181,0.001429,0.000563,0.001920
1321,0.111698,0.280702,0.012172,0.000494,0.000463,0.005665,0.000425,0.003500
1322,0.036696,0.372807,0.001593,0.000030,0.000033,0.000239,0.000085,0.000353


In [ ]:
# **